### read in ATTOM data

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# assessor data
df_assessor = pd.read_csv('../../ATTOM/1_Assessor/assessor_master.csv')
df_assessor = df_assessor[[
    '[ATTOM ID]',
    'lat',
    'lon',
    'year_built',
    'area_building'
]]

# sales (recorder) data
df_sales = pd.read_csv('../../ATTOM/2_Recorder/Recorder_master.csv')
df_sales = df_sales.drop_duplicates(subset='TransactionID', keep='first')
df_sales = df_sales[[
    '[ATTOM ID]',
    'TransferAmount',
    'sale_year'
]]

# create new dataframe that merges assessor and sales data
df_attom = pd.merge(df_assessor, df_sales, on='[ATTOM ID]', how='right')

# filters
df_attom = df_attom[df_attom['area_building'] <= 10000]
df_attom['price_sf'] = df_attom['TransferAmount'] / df_attom['area_building']

# drop any rows with null values in the lat/lon columns
df_attom = df_attom.dropna(subset=['lat', 'lon'])

# convert to geopandas GeoDataFrame using the lat/lon columns
df_attom_geo = gpd.GeoDataFrame(
    df_attom, 
    geometry=gpd.points_from_xy(df_attom['lon'], df_attom['lat']),
    crs="EPSG:4326"
    )

### read in other geographies

In [ ]:
# read in, clean up city data
cities = gpd.read_file('cities.geojson')
cities_to_keep = [
    'Acworth',
    'Atlanta',
    'Austell',
    'Avondale Estates',
    'Ball Ground',
    'Brookhaven',
    'Canton',
    'Chamblee',
    'Clarkston',
    'Conyers',
    'Cumming',
    'Dacula',
    'Decatur',
    'Doraville',
    'Douglasville',
    'Duluth',
    'Dunwoody',
    'East Point',
    'Fayetteville',
    'Forest Park',
    'Grayson',
    'Hampton',
    'Hapeville',
    'Johns Creek',
    'Jonesboro',
    'Kennesaw',
    'Lawrenceville',
    'Lithonia',
    'Locust Grove',
    'Marietta',
    'McDonough',
    'Milton',
    'Morrow',
    'Mountain Park',
    'Norcross',
    'Peachtree City',
    'Powder Springs',
    'Sandy Springs',
    'Smyrna',
    'Snellville',
    'South Fulton',
    'Stockbridge',
    'Stonecrest',
    'Sugar Hill',
    'Suwanee',
    'Union City',
]
cities = cities[['ShortLabel', 'geometry']]
cities_atl = cities[cities['ShortLabel'].isin(cities_to_keep)].reset_index(drop=True)
cities_atl = cities_atl.rename(columns={'ShortLabel': 'city'})

# read in, clean up county data
counties = gpd.read_file('../../Geographies/ARC_counties.gpkg')
counties['NAME'] = counties['NAME'].str.replace(' County', '')
counties = counties.rename(columns={'NAME': 'county'})

# read in, clean up tract data
tracts = gpd.read_file('../../Geographies/ARC_CTs.gpkg')
tracts = tracts[['GEOID', 'geometry']]
tracts = tracts.rename(columns={'GEOID': 'tract'})

# read in submarket data, merge with tracts
submarkets = pd.read_csv('HousingExplorer_data.csv')

tracts_with_submarkets = tracts.merge(
    submarkets, 
    left_on='tract', 
    right_on='Census Tract ID', 
    how='inner'
    ).drop(columns=['Census Tract ID'])

# set coordinate reference system
tracts_with_submarkets = tracts_with_submarkets.to_crs(epsg=4326)

### spatial joins & clean

In [ ]:
# set coordinate reference system of ATTOM to match other geographies
df_attom_geo = df_attom_geo.to_crs(epsg=4326)

# join ATTOM data to city, county, and tract data
df_attom_geo_join1 = gpd.sjoin(df_attom_geo, cities_atl, how='left', predicate='within').drop(columns=['index_right'])
df_attom_geo_join2 = gpd.sjoin(df_attom_geo_join1, counties, how='left', predicate='within').drop(columns=['index_right'])
df_final = gpd.sjoin(df_attom_geo_join2, tracts_with_submarkets, how='left', predicate='within').drop(columns=['index_right'])

# drop unnecessary columns
df_final = df_final.drop(columns='geometry')

# drop all rows with null values in 'county' column
df_final = df_final.dropna(subset=['county'])

### advanced cleaning steps using duplicate methodology, modified z-score

In [ ]:
# create shallow copy
df_clean = df_final.copy()

# Group by both ATTOM_ID and year to count occurrences for each group
grouped_counts_df = df_clean.groupby(
    ['[ATTOM ID]', 'sale_year']).size().reset_index(name='sale_count')

# Merge the grouped counts back into the original 'sales' DataFrame
df_clean = df_clean.merge(grouped_counts_df, on=['[ATTOM ID]', 'sale_year'], how='left')


# define function to compute the cleaned price / SF, which will calculate an "aggregated" price / sf if a property has sold more than once in a given year
def clean_price_sf(row):
    attom_id = row['[ATTOM ID]']
    year = row['sale_year']
    count = row['sale_count']

    if count == 1:
        return row['price_sf']
    else:
        subset = df_clean[(df_clean['[ATTOM ID]'] == attom_id)
                       & (df_clean['sale_year'] == year)]
        if count <= 3:
            return subset['price_sf'].max()
        else:
            return subset['price_sf'].median()

# just keep 2023 and 2024
df_clean = df_clean[df_clean['sale_year'].isin([2019, 2024])]

# Apply the de-dupe function defined above to create the 'price_sf_cleaned' column
df_clean = df_clean.copy()

print('applying the clean_price_sf function to the df_clean dataframe')
df_clean['price_sf_cleaned'] = df_clean.apply(clean_price_sf, axis=1)

# 2-step to drop the original 'price_sf' column and then rename the cleaned 'price_sf' back to its original name
df_clean = df_clean.drop(columns='price_sf').rename(columns={
    'price_sf_cleaned': 'price_sf'
})


# define the modified z-score outliers function
def filter_by_modified_zscore(data, threshold=3.5):
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    if mad == 0:
        return data  # No outliers if mad is 0
    modified_z_scores = 0.6745 * (data - median) / mad
    return data[np.abs(modified_z_scores) < threshold]


# define the percentile outliers filter function
# NOTE: this is not used in the final output, but is included for reference
def filter_by_percentiles(data, lower_percentile, upper_percentile):
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return data[(data >= lower_bound) & (data <= upper_bound)]


# define function to filter data by each method
def filter_group(group, method, **kwargs):
    prices = group['price_sf']
    if method == 'modified_zscore':
        filtered_prices = filter_by_modified_zscore(prices, **kwargs)
    elif method == 'percentiles':
        filtered_prices = filter_by_percentiles(prices, **kwargs)
    return group[group['price_sf'].isin(filtered_prices)]


sales_years = list(df_clean['sale_year'].unique())

sales_list = []
for year in sales_years:
    df_year = df_clean[df_clean['sale_year'] == year]

    filtered_sales_zscore = df_year.groupby('GEOID', group_keys=False).apply(
        filter_group,
        method='modified_zscore',
        threshold=3.5,
        include_groups=False
    ).reset_index(drop=True)

    sales_list.append(filtered_sales_zscore)

# create cleaned dataframe, inclusive of all years
combined_cleaned_full_df = pd.concat(sales_list, ignore_index=False)

applying the clean_price_sf function to the df_clean dataframe


### perform aggregations by geo level

In [ ]:
# in the Submarket column, replace 'Atlanta' with 'Atlanta-Sandy Springs-Marietta'
combined_cleaned_full_df['Submarket'] = combined_cleaned_full_df['Submarket'].astype(str).str.replace('.0', '')
combined_cleaned_full_df['Submarket_name'] = 'Submarket ' + combined_cleaned_full_df['Submarket'] 

# drop the Submarket column
combined_cleaned_full_df = combined_cleaned_full_df.drop(columns=['Submarket'])

# create 2019 and 2024 dataframes
df_2019 = combined_cleaned_full_df[combined_cleaned_full_df['sale_year'] == 2019]
df_2024 = combined_cleaned_full_df[combined_cleaned_full_df['sale_year'] == 2024]

# 2019 data ------------------------------------------------------------
# --- City-level aggregation ---
city_df = (
    df_2019[df_2019['city'].notna()]
    .groupby('city')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median')
    )
    .reset_index()
    .rename(columns={'city': 'name'})
)
city_df['type'] = 'city'

# --- County-level aggregation ---
county_df = (
    df_2019.groupby('county')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median')
    )
    .reset_index()
    .rename(columns={'county': 'name'})
)
county_df['type'] = 'county'

# --- Submarket-level aggregation ---
submarket_df = (
    df_2019.groupby('Submarket_name')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median')
    )
    .reset_index()
    .rename(columns={'Submarket_name': 'name'})
)
submarket_df['type'] = 'submarket'

# --- Regional-level aggregation ---
region_data = {
    'name': 'Atlanta Region',
    'type': 'region',
    'median_year_built': df_2019['year_built'].median(),
    'median_area': df_2019['area_building'].median(),
    'median_sale_price': df_2019['TransferAmount'].median(),
    'median_price_sf': df_2019['price_sf'].median()
}
region_df = pd.DataFrame([region_data])

# --- Combine all aggregations ---
summary_df_2019 = pd.concat([city_df, county_df, submarket_df, region_df], ignore_index=True)

# --- Final column ordering (optional but clean) ---
summary_df_2019 = summary_df_2019[['name', 'type', 'median_year_built', 'median_area', 'median_sale_price', 'median_price_sf']]

# export
summary_df_2019.to_csv('ATTOM_2019.csv', index=False)

# 2024 data ------------------------------------------------------------
# --- City-level aggregation ---
city_df = (
    df_2024[df_2024['city'].notna()]
    .groupby('city')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median'),
        total_sales=('TransferAmount', 'count')
    )
    .reset_index()
    .rename(columns={'city': 'name'})
)
city_df['type'] = 'city'

# --- County-level aggregation ---
county_df = (
    df_2024.groupby('county')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median'),
        total_sales=('TransferAmount', 'count')
    )
    .reset_index()
    .rename(columns={'county': 'name'})
)
county_df['type'] = 'county'

# --- Census tract-level aggregation ---
tract_df = (
    df_2024.groupby('tract')
    .agg(
        median_year_built=('year_built', 'median'),
        median_area=('area_building', 'median'),
        median_sale_price=('TransferAmount', 'median'),
        median_price_sf=('price_sf', 'median'),
        total_sales=('TransferAmount', 'count')
    )
    .reset_index()
    .rename(columns={'tract': 'name'})
)
tract_df['type'] = 'tract'

# --- Regional-level aggregation ---
region_data = {
    'name': 'Atlanta Region',
    'type': 'region',
    'median_year_built': df_2024['year_built'].median(),
    'median_area': df_2024['area_building'].median(),
    'median_sale_price': df_2024['TransferAmount'].median(),
    'median_price_sf': df_2024['price_sf'].median(),
    'total_sales': df_2024['TransferAmount'].count()
}
region_df = pd.DataFrame([region_data])

# --- Combine all aggregations ---
summary_df_2024 = pd.concat([city_df, county_df, tract_df, region_df], ignore_index=True)

# --- Final column ordering (optional but clean) ---
summary_df_2024 = summary_df_2024[['name', 'type', 'total_sales', 'median_year_built', 'median_area', 'median_sale_price', 'median_price_sf']]

# export
summary_df_2024.to_csv('ATTOM_2024.csv', index=False)